In [1]:
from pycaret.nlp import *
from pycaret.classification import *

import warnings
# warnings.filterwarnings('ignore')
from common.preprocess import Preprocess

import nltk
%time nltk.download('wordnet')


CPU times: user 82 µs, sys: 22 µs, total: 104 µs
Wall time: 107 µs


[nltk_data] Downloading package wordnet to /home/alain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# !pip install pycaret

In [3]:
from nltk.corpus import stopwords
from common.preprocess import Preprocess

prep = Preprocess()

def init_process(frac) -> pd.DataFrame:
    df = pd.read_csv("data/dataset.csv", sep = ";")

    df['description'] = df['description'].apply(lambda x : prep.clean_data(x))
    df = prep.cleandataset(df)

    # check class distribution
    print(df['fraudulent'].value_counts(normalize = True))

    df_fraud = df[df.fraudulent == 1]
    df_true = df[df.fraudulent == 0]

    # Split the df_true to re balance the dataset
    print(f"\nfrac = {frac}")
    df_true_sample = df_true.sample(frac=frac, random_state=0)

    df_reshape = pd.concat([df_fraud, df_true_sample])

    return df_reshape

%time df = init_process(0.5)

0    0.95731
1    0.04269
Name: fraudulent, dtype: float64
CPU times: user 21.5 s, sys: 475 ms, total: 22 s
Wall time: 22 s


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer


# BoW-TF:IDF Embedding
tfidf_vectorizer = TfidfVectorizer(min_df=.02,
                                    max_df=.7,
                                    ngram_range=[1,3],
                                    max_features=50000,
                                    lowercase=False)

tpl_tfidf = tfidf_vectorizer.fit_transform(df['description'])
display(f"Bow-TF:IDF : {tpl_tfidf.shape}")

df_tfidf = pd.DataFrame(tpl_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names(), index=df.index)
df_tfidf.head()


'Bow-TF:IDF : (14500, 1200)'

,000,10,100,12,15,20,2014,30,40,50,500,ability,ability work,able,able work,about,access,according,account,accounting,accounts,accuracy,accurate,achieve,acquisition,across,act,action,active,actively,activities,activity,ad,add,addition,additional,address,adhere,administration,administrative,advanced,advertising,agencies,agency,agile,all,allow,along,alongside,already,also,always,amazing,ambitious,an,analysis,analyst,analytical,analytics,analyze,and,and or,android,annual,answer,answering,app,applicant,application,applications,apply,apply now,approach,appropriate,apps,architecture,are,area,areas,around,art,as,aspect,asset,assigned,assignment,assist,assistance,assistant,assisting,associate,assurance,at,attend,attention,attention detail,attitude,audience,automated,automation,available,award,award winning,awareness,awesome,back,backed,background,balance,bank,base,based,basic,basis,be,beautiful,become,believe,benefit,benefits,best,best practice,better,big,billing,board,bonus,brand,bring,budget,build,building,built,business,business development,by,call,calls,campaign,campaigns,can,candidate,candidate integral,candidate integral part,candidate must,candidates,capability,capital,care,career,case,cash,center,ceo,challenge,challenging,chance,change,changing,channel,charge,check,city,class,clean,clear,click,client,clients,close,closely,closing,cloud,co,code,coding,collaborate,collaboration,collaborative,come,comfortable,commercial,commitment,committed,communicate,communication,communication skill,communication skills,communications,community,companies,company,compensation,competitive,complete,completion,complex,compliance,component,comprehensive,computer,concept,conduct,confidential,considered,consistent,consistently,constantly,consultant,consulting,consumer,contact,content,continued,continued growth,continuous,contract,contribute,control,coordinate,coordination,copy,core,corporate,cost,could,country,cover,create,creating,creation,creative,credit,critical,crm,cross,cross functional,css,culture,current,currently,currently seeking,custom,customer,customer care,customer service,customers,cutting,cutting edge,cycle,...,self motivated,self starter,sell,selling,send,senior,sense,sensitive,serve,server,service,services,serving,set,setting,several,share,shipping,short,show,side,significant,simple,since,site,size,skill,skilled,skills,small,small team,smart,so,social,social media,social medium,software,software development,solid,solution,solutions,solve,solving,someone,something,sound,source,space,special,specialist,specific,specification,sql,stack,staff,stage,stakeholder,standard,standards,start,start up,starter,starting,startup,state,states,status,stay,store,story,strategic,strategies,strategy,strong,student,submit,success,successful,successful candidate,successfully,suite,summary,supervision,supervisor,supply,support,supporting,supporting continued,supporting continued growth,sure,system,systems,take,taking,talent,talented,talented team,talented team supporting,target,task,tasks,team,team member,team members,team player,team supporting,team supporting continued,team we,team you,teams,tech,technical,technologies,technology,telephone,term,test,testing,that,the,the company,the customer,the ideal,the ideal candidate,the position,the right,the right candidate,the role,the successful,the successful candidate,them,there,they,thing,think,thinking,this,this position,this role,thousand,three,throughout,time,timely,times,title,to,to day,today,together,tool,tools,top,touch,towards,track,track record,tracking,train,training,travel,trend,troubleshooting,two,type,ui,uk,understand,understanding,unique,unit,united,up,update,upon,us,use,used,user,user experience,users,using,ux,value,variety,various,ve,vendor,venture,verbal,via,video,view,vision,visit,visual,volume,want,way,we,we believe,we currently,we looking,we need,we offer,we provide,we re,we re looking,we seeking,we ve,web,website,week,weekly,well,what,who,wide,will,willing,winning,with,within,without,word,work

In [5]:
#Preparing processed and BoW-TF:IDF embedded data for Classification
df_tfidf_m = pd.concat([df, df_tfidf], axis = 1)
df_tfidf_m.drop(columns=['description'], inplace = True)
print(df_tfidf_m.shape)
df_tfidf_m.head()

(14500, 1200)


,fraudulent,000,10,100,12,15,20,2014,30,40,50,500,ability,ability work,able,able work,about,access,according,account,accounting,accounts,accuracy,accurate,achieve,acquisition,across,act,action,active,actively,activities,activity,ad,add,addition,additional,address,adhere,administration,administrative,advanced,advertising,agencies,agency,agile,all,allow,along,alongside,already,also,always,amazing,ambitious,an,analysis,analyst,analytical,analytics,analyze,and,and or,android,annual,answer,answering,app,applicant,application,applications,apply,apply now,approach,appropriate,apps,architecture,are,area,areas,around,art,as,aspect,asset,assigned,assignment,assist,assistance,assistant,assisting,associate,assurance,at,attend,attention,attention detail,attitude,audience,automated,automation,available,award,award winning,awareness,awesome,back,backed,background,balance,bank,base,based,basic,basis,be,beautiful,become,believe,benefit,benefits,best,best practice,better,big,billing,board,bonus,brand,bring,budget,build,building,built,business,business development,by,call,calls,campaign,campaigns,can,candidate,candidate integral,candidate integral part,candidate must,candidates,capability,capital,care,career,case,cash,center,ceo,challenge,challenging,chance,change,changing,channel,charge,check,city,class,clean,clear,click,client,clients,close,closely,closing,cloud,co,code,coding,collaborate,collaboration,collaborative,come,comfortable,commercial,commitment,committed,communicate,communication,communication skill,communication skills,communications,community,companies,company,compensation,competitive,complete,completion,complex,compliance,component,comprehensive,computer,concept,conduct,confidential,considered,consistent,consistently,constantly,consultant,consulting,consumer,contact,content,continued,continued growth,continuous,contract,contribute,control,coordinate,coordination,copy,core,corporate,cost,could,country,cover,create,creating,creation,creative,credit,critical,crm,cross,cross functional,css,culture,current,currently,currently seeking,custom,customer,customer care,customer service,customers,cutting,cutting edge,...,self motivated,self starter,sell,selling,send,senior,sense,sensitive,serve,server,service,services,serving,set,setting,several,share,shipping,short,show,side,significant,simple,since,site,size,skill,skilled,skills,small,small team,smart,so,social,social media,social medium,software,software development,solid,solution,solutions,solve,solving,someone,something,sound,source,space,special,specialist,specific,specification,sql,stack,staff,stage,stakeholder,standard,standards,start,start up,starter,starting,startup,state,states,status,stay,store,story,strategic,strategies,strategy,strong,student,submit,success,successful,successful candidate,successfully,suite,summary,supervision,supervisor,supply,support,supporting,supporting continued,supporting continued growth,sure,system,systems,take,taking,talent,talented,talented team,talented team supporting,target,task,tasks,team,team member,team members,team player,team supporting,team supporting continued,team we,team you,teams,tech,technical,technologies,technology,telephone,term,test,testing,that,the,the company,the customer,the ideal,the ideal candidate,the position,the right,the right candidate,the role,the successful,the successful candidate,them,there,they,thing,think,thinking,this,this position,this role,thousand,three,throughout,time,timely,times,title,to,to day,today,together,tool,tools,top,touch,towards,track,track record,tracking,train,training,travel,trend,troubleshooting,two,type,ui,uk,understand,understanding,unique,unit,united,up,update,upon,us,use,used,user,user experience,users,using,ux,value,variety,various,ve,vendor,venture,verbal,via,video,view,vision,visit,visual,volume,want,way,we,we believe,we currently,we looking,we need,we offer,we provide,we re,we re looking,we seeking,we ve,web,website,week,weekly,well,what,who,wide,will,willing,winning,with,within,without,word

In [6]:
# df_tfidf_m_columns = df_tfidf_m.columns
# df_tfidf_m_columns
# res = [sub for sub in df_tfidf_m_columns if sub.isalpha()]
# res.append('fraudulent')
# df_tfidf_m = df_tfidf_m.drop(columns=[col for col in df_tfidf_m if col not in res])
# df_tfidf_m.head()

In [7]:
%time pce_1 = setup(data = df_tfidf_m, target = 'fraudulent', session_id = 5, train_size = 0.8, use_gpu=True, fix_imbalance=True, preprocess=False)

,Description,Value
0,session_id,5
1,Target,fraudulent
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14500, 1200)"
5,Missing Values,False
6,Numeric Features,1199
7,Categorical Features,0
8,Transformed Train Set,"(11600, 1199)"
9,Transformed Test Set,"(2900, 1199)"


CPU times: user 32.4 s, sys: 15.4 s, total: 47.8 s
Wall time: 1min 8s


In [8]:
%time compare_models(sort = 'F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9705,0.8905,0.3000,0.9207,0.4449,0.4342,0.5098,4.8830
lda,Linear Discriminant Analysis,0.9560,0.8162,0.3553,0.4535,0.3969,0.3745,0.3783,5.5580
et,Extra Trees Classifier,0.9692,0.8885,0.2447,0.9808,0.3869,0.3773,0.4775,3.6930
rf,Random Forest Classifier,0.9676,0.8685,0.2043,0.9756,0.3304,0.3216,0.4293,4.5980
dt,Decision Tree Classifier,0.9480,0.6439,0.2957,0.3363,0.3133,0.2866,0.2879,11.8110
knn,K Neighbors Classifier,0.9660,0.7407,0.1787,0.9153,0.2967,0.2874,0.3931,19.7100
gbc,Gradient Boosting Classifier,0.9634,0.8156,0.1447,0.7415,0.2378,0.2283,0.3119,33.2940
svm,SVM - Linear Kernel,0.9647,0.0000,0.1340,0.9800,0.2298,0.2227,0.3445,0.4060
ada,Ada Boost Classifier,0.9583,0.8125,0.1362,0.4624,0.2094,0.1943,0.2340,7.1480
nb,Naive Bayes,0.7336,0.7768,0.7766,0.1092,0.1914,0.1295,0.2214,0.1830


CPU times: user 25min 30s, sys: 3min 2s, total: 28min 32s
Wall time: 16min 24s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [9]:
#step1 : model creation
%time pce_1_m1 = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9707,0.8643,0.3191,0.8824,0.4688,0.4571,0.5207
1,0.9750,0.9251,0.4255,0.9091,0.5797,0.5686,0.6125
2,0.9741,0.9047,0.3617,1.0000,0.5312,0.5209,0.5935
3,0.9707,0.8956,0.2979,0.9333,0.4516,0.4406,0.5183
4,0.9724,0.8768,0.3191,1.0000,0.4839,0.4735,0.5570
5,0.9647,0.8766,0.1277,1.0000,0.2264,0.2193,0.3509
6,0.9681,0.9168,0.2340,0.9167,0.3729,0.3624,0.4543
7,0.9690,0.8571,0.2553,0.9231,0.4000,0.3893,0.4765
8,0.9655,0.8655,0.2340,0.7333,0.3548,0.3419,0.4022


CPU times: user 6min 37s, sys: 1.14 s, total: 6min 38s
Wall time: 55.9 s


In [15]:
#step2 : model tuning
%time tuned_pce_1_m1 = tune_model(pce_1_m1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9707,0.8434,0.3191,0.8824,0.4688,0.4571,0.5207
1,0.9733,0.9100,0.3830,0.9000,0.5373,0.5258,0.5774
2,0.9698,0.8846,0.2766,0.9286,0.4262,0.4154,0.4978
3,0.9681,0.8826,0.2766,0.8125,0.4127,0.4004,0.4630
4,0.9707,0.8753,0.2979,0.9333,0.4516,0.4406,0.5183
5,0.9681,0.8955,0.2128,1.0000,0.3509,0.3415,0.4538
6,0.9664,0.8970,0.2553,0.7500,0.3810,0.3679,0.4255
7,0.9690,0.8492,0.2766,0.8667,0.4194,0.4077,0.4796
8,0.9690,0.8676,0.2979,0.8235,0.4375,0.4251,0.4843


CPU times: user 5min 5s, sys: 1.52 s, total: 5min 6s
Wall time: 5min 58s


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns

In [18]:
#step3 : getting insights from model performance
%time eval = evaluate_model(tuned_pce_1_m1)
eval

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

CPU times: user 614 ms, sys: 0 ns, total: 614 ms
Wall time: 368 ms
